In [1]:
import pandas as pd
import processDataset
import fuzzywuzzy
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
# Reading in RecipeNLG
recipe_nlg = pd.read_csv("full_dataset.csv")

In [3]:
recipe_dict = processDataset.process_df_recipe(recipe_nlg)

In [4]:
recipe_dict = processDataset.filter_units_recipe(recipe_dict)

In [5]:
recipe_dict = processDataset.standardize_units_recipe(recipe_dict)

In [11]:
# final filter
acceptable_units = ["cup", "teaspoon", "tablespoon", "gallon", "c.", "Tbsp", "tbsp", "tsp", "can", "gal.", "teaspoon", "pt.", "pint", "stick", "quart", "qt."]
unacceptable = set()
for recipe in recipe_dict.keys():
    accept = False
    for ingredient in recipe_dict[recipe][5].split(" "):
        if ingredient in acceptable_units:
            accept = True
    if not accept:
        unacceptable.add(recipe)
for rec in unacceptable:
    del(recipe_dict[rec])

In [12]:
nutrients = pd.read_csv("nutrition.csv")

In [13]:
nutrient_dict = processDataset.process_df_nutrients(nutrients)

In [14]:
from fuzzywuzzy import fuzz

In [15]:
recipe_to_ingredient = {}
for key in recipe_dict.keys():
    matches = []
    if recipe_dict[key][2] == ['']:
        continue
    for recipe_ingredient in recipe_dict[key][2]:
        max_match = 0
        match = [None, None]
        for ingredient in nutrient_dict.keys():
            curr_match = fuzz.token_set_ratio(recipe_ingredient, ingredient)
            if curr_match >= 90 and curr_match > max_match:
                max_match = curr_match
                match = [recipe_ingredient, ingredient]
        if max_match == 0:
            break
        matches.append(match)
    if len(matches) == len(recipe_dict[key][2]):
        recipe_to_ingredient[key] = []
        for match in matches:
            recipe_to_ingredient[key].append(match)

that took a while lol

In [16]:
import csv

In [17]:
with open('pairs.csv', 'w', newline='') as csvfile:
    header = ['key'] + [str(i) for i in range(100)]
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for key in recipe_to_ingredient.keys():
        row = {}
        row['key'] = key
        for a in range(len(recipe_to_ingredient[key])):
            row[str(2 * a)] = recipe_to_ingredient[key][a][0]
            row[str((2 * a) + 1)] = recipe_to_ingredient[key][a][1]
        writer.writerow(row)



In [18]:
mappings = pd.read_csv("pairs.csv")

/var/folders/0z/cssgr2ps5mx79h2qty5lxfmr0000gn/T/ipykernel_2076/4251665067.py:1: DtypeWarning: Columns (37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  mappings = pd.read_csv("pairs.csv")


In [19]:
recipe_to_ingredient = processDataset.process_pairs(pd, mappings)

In [25]:
recipe_info = {}
for key in recipe_to_ingredient.keys():
    nutritionInfo = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    quantity = 0.0
    for pair in recipe_to_ingredient[key]:
        recipe = pair[0]
        ingredient = pair[1]
        for a in range(len(recipe_dict[key][2])):
            if recipe_dict[key][2][a] == recipe:
                quantity += recipe_dict[key][0][a]
                ratio = recipe_dict[key][0][a] / nutrient_dict[ingredient][0]
                for b in range(1, len(nutritionInfo)):
                    nutritionInfo[b - 1] += nutrient_dict[ingredient][b] * ratio
                continue
    recipe_info[key] = {
        'calories': str(round(nutritionInfo[0], 2)),
        'fat': str(round(nutritionInfo[1], 2)),
        'saturated_fat': str(round(nutritionInfo[2], 2)),
        'cholesterol': str(round(nutritionInfo[3], 2)),
        'protein': str(round(nutritionInfo[4], 2)),
        'carbohydrates': str(round(nutritionInfo[5], 2)),
        'fiber': str(round(nutritionInfo[6], 2)) ,
        'sugar': str(round(nutritionInfo[7], 2)),
        'caffeine': str(round(nutritionInfo[8], 2)),
        'ingredients': recipe_dict[key][5],
        'quantity': str(round(quantity, 2)),
        'directions': recipe_dict[key][3],
        'link': recipe_dict[key][4]
    }    

In [26]:
print(len(recipe_info.keys()))

27593


In [27]:
with open('final_dataset.csv', 'w', newline='') as csvfile:
    header = ['name', 'ingredients', 'directions', 'calories', 'fat', 'saturated_fat', 'cholesterol', 'protein', 'carbohydrates', 'fiber', 'sugar', 'caffeine', 'quantity', 'link']
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for key in recipe_info.keys():
        row = recipe_info[key]
        row['name'] = key
        writer.writerow(row)